Creating a SQLite3 database of earthquake travel-time data

Files begin in input format for struct3dp inversion program

First read in the travel time data from the different files

In [1]:
import pandas as pd
import numpy as np

# define the directory that has the struct3dp data files
modelname='Run_230'
rundir='/Users/ulberg/research/MSH/crosson/Runs'
datadir='{}/{}/Data'.format(rundir,modelname)

# These file names are always associated with an inversion
stafile='combined.sta' # the stations that recorded travel times
eqobsfile='quakes.obs' # the travel time observations for earthquakes
eqlocfile='quakes.loc' # the locations of earthquakes
exobsfile='explos.obs' # the travel time observations for explosions
exlocfile='explos.loc' # the locations of explosions

sta=pd.read_csv(datadir + '/' + stafile, names=['Station','Longitude','Latitude','Depth'], sep=' ', skipinitialspace=True)
obs_eq=pd.read_csv(datadir + '/' + eqobsfile, names=['SourceID','Station','Phase','ArrivalTime','Uncertainty'], sep=' ', skipinitialspace=True)
loc_eq=pd.read_csv(datadir + '/' + eqlocfile, names=['SourceID','Longitude','Latitude','Depth','EventTime'], sep=' ', skipinitialspace=True)
obs_ex=pd.read_csv(datadir + '/' + exobsfile, names=['SourceID','Station','Phase','ArrivalTime','Uncertainty'], sep=' ', skipinitialspace=True)
loc_ex=pd.read_csv(datadir + '/' + exlocfile, names=['SourceID','Longitude','Latitude','Depth','EventTime'], sep=' ', skipinitialspace=True)


Also read in files with information relating PNSN events to Antelope events (two different methods of locating earthquakes)

In [2]:
mapfile='map.map'
antUWfile='AntUWreview.txt'

mapcols=['dbname','orid','SourceID','method','Longitude-ant','Latitude-ant','Depth-ant','FullTime']
antUWcols=['PNSNid','dbname','orid','picker','Latitude-uw','Longitude-uw','Depth-uw']

datamap=pd.read_csv(datadir + '/' + mapfile, names=mapcols, sep=' ', skipinitialspace=True)
# antUW=pd.read_csv(rundir + '/' + modelname + '/ANT/' + antUWfile, names=antUWcols, sep=' ', skipinitialspace=True)
antUW=pd.read_csv('{}/{}/ANT/{}'.format(rundir,modelname,antUWfile), names=antUWcols, sep=' ', skipinitialspace=True, dtype={'orid': str})
antUW['orid']=antUW['orid'].astype('object') # so that merging later will work
antUW['PNSNid']=antUW['PNSNid'].astype('object') # so that merging later will work

We need to relate the source id's to the antelope database so we can get a full origin time for the event. Start with the ones that were recorded on the iMUSH broadband instruments (SourceID starts with '4', '5', '6')

In [3]:
# sources were organized based on the first 1 or 2 digits of the id
# define what the initial character means
src_antelope=['4','5','6']
dict_ant={}
for k in src_antelope:
    for j in range(1,5):
        dict_ant[k+str(j)]='201' + k + '_Q' + str(j)
        
src_pnsn=['9']
src_shot=['8'] # or anything else

# read in the first and second characters in the SourceID, this bit could use some cleaning
srcID0=loc_eq['SourceID'].apply(lambda x: x[0])
# srcID1=loc_eq['SourceID'].apply(lambda x: x[1])
# srcID01=loc_eq['SourceID'].apply(lambda x: x[:2])
# print('Data types: ' + str(srcID0.unique()))

# is it from antelope?
# isAnt=srcID0.apply(lambda x: x in src_antelope)
loc_eq['dbname']=loc_eq['SourceID'].apply(lambda x: x[:2]).map(dict_ant)
# loc_eq.dbname.value_counts() # how many events from each quarter are there?

# if the source is in antelope, get the antelope orid (This could also be done with datamap, or doesn't have to be done here at all)
loc_eq['SourceID'][loc_eq['dbname'].notnull()]
loc_eq['orid']=loc_eq[['SourceID','dbname']].apply(lambda x: x[0][-5:-1] if pd.notnull(x[1]) else 'NaN', axis=1)

Add the full origin time from datamap to loc_eq, plus pnsn location

In [4]:
df=loc_eq.merge(datamap[['SourceID','dbname','orid','FullTime']],left_on=['SourceID','dbname','orid'],right_on=['SourceID','dbname','orid'],how='left')
# df['orid']=df['orid'].astype('object')
# df['dbname']=df['dbname'].astype('object')
# antUW['orid']=antUW['orid'].astype('object')
# antUW['dbname']=antUW['dbname'].astype('object')
df2=df.merge(antUW[['PNSNid','dbname','orid','Latitude-uw','Longitude-uw','Depth-uw']],left_on=['dbname','orid'],right_on=['dbname','orid'],how='left')
df2.head()

,SourceID,Longitude,Latitude,Depth,EventTime,dbname,orid,FullTime,PNSNid,Latitude-uw,Longitude-uw,Depth-uw
0,4301394p,-122.4719,45.8537,0.000,26.897,2014_Q3,1394,20140701T222626.897,60810642,45.8663,-122.4492,0.0
1,4301395p,-122.6557,45.5633,15.500,41.314,2014_Q3,1395,20140703T132841.314,60058633,45.5633,-122.6557,15.9
2,4301396p,-122.4267,46.1724,0.123,28.559,2014_Q3,1396,20140709T181928.559,60814162,46.1728,-122.4225,0.0
3,4301401p,-122.4558,46.3526,1.420,30.893,2014_Q3,1401,20140714T173030.893,0,46.3526,-122.4558,1.4
4,4301404p,-121.9153,46.8328,10.400,52.527,2014_Q3,1404,20140715T135452.527,60818252,46.8328,-121.9153,11.8


In [5]:
## check lengths of df (merged), loc_eq (s3dp), datamap (antelope), antUW (pnsn)
print('df: {}'.format(len(df)))
print('df2: {}'.format(len(df2)))
print('loc_eq: {}'.format(len(loc_eq)))
print('datamap: {}'.format(len(datamap)))
print('antUW: {}'.format(len(antUW)))

df: 899
df2: 899
loc_eq: 899
datamap: 2177
antUW: 449


In [92]:
import time
## adjust the observation dataframe to make the observation time the full time - use df to do this
# full observation = df.FullTime + (obs_eq.ArrivalTime - df.EventTime) 
# make sure units are correct, the ArrivalTime and EventTime are in seconds
# also need to make sure all of the df rows have a FullTime - currently the PNSN ones do not?
print("obs_eq")
print(obs_eq.head())
print("df")
print(df.head())

#### loop over rows efficiently (or not efficiently)
### 1st version ~ 65 seconds
def ftmp(SourceID,ArrivalTime):
    row=df[df['SourceID']==SourceID]
    newTime=pd.to_datetime(row['FullTime']) + pd.Timedelta(seconds=ArrivalTime) - pd.to_timedelta(row['EventTime'], unit='s')
    return newTime
# start_time = time.time()
# res = [ftmp(x, y) for x, y in zip(obs_eq['SourceID'], obs_eq['ArrivalTime'])] # ~65 seconds
# elapsed_time = time.time() - start_time
# elapsed_time

###  2nd version ~ 7 seconds
##   create new column in obs_eq that is the index to the corresponding event in df
# Example:
# a=pd.DataFrame(data={'A':['a','a','b','c','c']})
# b=pd.DataFrame(data={'A':['a','b','c']})
# inds=[np.where(b['A']==a['A'][i])[0][0] for i in range(len(a))]
start_time = time.time()
inds=[np.where(df['SourceID']==obs_eq['SourceID'][i])[0][0] for i in range(len(obs_eq))] # ~6 seconds
elapsed_time = time.time() - start_time
print('Indexing: ' + str(elapsed_time))
# full observation = df.FullTime + (obs_eq.ArrivalTime - df.EventTime) 
FullArrivalTime=pd.to_datetime(df.iloc[inds]['FullTime']).values + pd.to_timedelta(obs_eq.ArrivalTime,unit='s').values - pd.to_timedelta(df.iloc[inds]['EventTime'],unit='s').values
elapsed_time = time.time() - start_time
print('Full 2nd version: ' + str(elapsed_time))

obs_eq['FullArrivalTime']=FullArrivalTime


obs_eq
   SourceID Station Phase  ArrivalTime  Uncertainty
0  4301394p    MO06     P       31.432        0.096
1  4301394p    MM02     P       32.135        0.080
2  4301394p    MO07     P       32.552        0.081
3  4301394p    MN07     P       33.230        0.121
4  4301394p    MJ02     P       33.790        0.092
df
   SourceID  Longitude  Latitude   Depth  EventTime   dbname  orid  \
0  4301394p  -122.4719   45.8537   0.000     26.897  2014_Q3  1394   
1  4301395p  -122.6557   45.5633  15.500     41.314  2014_Q3  1395   
2  4301396p  -122.4267   46.1724   0.123     28.559  2014_Q3  1396   
3  4301401p  -122.4558   46.3526   1.420     30.893  2014_Q3  1401   
4  4301404p  -121.9153   46.8328  10.400     52.527  2014_Q3  1404   

              FullTime  
0  20140701T222626.897  
1  20140703T132841.314  
2  20140709T181928.559  
3  20140714T173030.893  
4  20140715T135452.527  
Indexing: 6.186533212661743
Full 2nd version: 6.3811352252960205


In [96]:
# testing 2nd version above, originally didn't work because indexing for FullTime and EventTime were different than ArrivalTime (because they came from df)?

# len(obs_eq)
# print(len(FullArrivalTime))
# len(inds)
# FullTime=pd.to_datetime(df.iloc[inds]['FullTime']).values
# ArrivalTime=pd.to_timedelta(obs_eq.ArrivalTime,unit='s').values
# EventTime=pd.to_timedelta(df.iloc[inds]['EventTime'],unit='s').values
# FullArrivalTime2=FullTime+ArrivalTime-EventTime
# print(len(FullTime+ArrivalTime))
# print(len(FullTime-EventTime))
# print(len(ArrivalTime-EventTime))
# # FullArrivalTime=pd.to_datetime(df.iloc[inds]['FullTime']) + pd.to_timedelta(obs_eq.ArrivalTime,unit='s') - pd.to_timedelta(df.iloc[inds]['EventTime'],unit='s')
# print('{}, {}, {}'.format(len(pd.to_datetime(df.iloc[inds]['FullTime'])),len(pd.to_timedelta(obs_eq.ArrivalTime,unit='s')),len(pd.to_timedelta(df.iloc[inds]['EventTime'],unit='s'))))
# print('{}, {}, {}'.format(len(FullTime),len(ArrivalTime),len(EventTime)))
# print('{}, {}, {}'.format(type(FullTime),type(ArrivalTime),type(EventTime)))
# print(len(FullArrivalTime2))
# # print(2079229/63145)

In [97]:
obs_eq

,SourceID,Station,Phase,ArrivalTime,Uncertainty,FullArrivalTime
0,4301394p,MO06,P,31.432,0.096,2014-07-01 22:26:31.432
1,4301394p,MM02,P,32.135,0.080,2014-07-01 22:26:32.135
2,4301394p,MO07,P,32.552,0.081,2014-07-01 22:26:32.552
3,4301394p,MN07,P,33.230,0.121,2014-07-01 22:26:33.230
4,4301394p,MJ02,P,33.790,0.092,2014-07-01 22:26:33.790
5,4301394p,MI05,P,35.144,0.157,2014-07-01 22:26:35.144
6,4301394p,MH03,P,35.635,0.067,2014-07-01 22:26:35.635
7,4301394p,MG06,P,37.419,0.242,2014-07-01 22:26:37.419
8,4301394p,MM04,P,31.051,0.047,2014-07-01 22:26:31.051
9,4301395p,MP04,P,48.055,0.102,2014-07-03 13:28:48.055


Now put this into a sql database

In [ ]:
#### one version, see below for another
# following https://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html

import sqlite3
import os

def addStationRow(rowNum,df):
    # column names are already defined as [name,lat,lon,depth]
    c.execute("INSERT INTO {tn} ({idf}, {cn1}, {cn2}, {cn3}) VALUES ('{v0}', {v1}, {v2}, {v3})".\
         format(tn='station', idf='name', cn1='lat', cn2='lon', cn3='depth', v0=sta.iloc[rowNum]['Station'],\
               v1=sta.iloc[rowNum]['Latitude'],v2=sta.iloc[rowNum]['Longitude'],v3=sta.iloc[rowNum]['Depth']))

dbdir='/Users/ulberg/Documents/GitHub/travelTimeDB/DB' # make sure this directory is created before running
filename=dbdir + '/tt_db.sqlite'

# remove database to start (DANGEROUS)
if os.path.exists(filename):
    os.remove(filename)
else:
    print("The file does not exist")

# open a connection to the database
conn=sqlite3.connect(filename)
c=conn.cursor()

tableA='station'
fieldA1='name'
typeA1='TEXT'

fieldA2='lat'
typeA2='REAL'

# creates table and adds column names, using different methods for testing
c.execute("CREATE TABLE {tn} ({cn} {ct} PRIMARY KEY)".format(tn=tableA,cn=fieldA1,ct=typeA1))
c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}".format(tn=tableA,cn=fieldA2,ct=typeA2))
c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}".format(tn=tableA,cn='lon',ct='REAL'))
c.execute("ALTER TABLE {tn} ADD COLUMN '{cn}' {ct}".format(tn=tableA,cn='depth',ct='REAL'))

# inserts one row (for string entry, need extra quotes around it - '{v0}')
# c.execute("INSERT INTO {tn} ({idf}, {cn1}, {cn2}, {cn3}) VALUES ('{v0}', {v1}, {v2}, {v3})".\
#          format(tn=tableA, idf=fieldA1, cn1='lat', cn2='lon', cn3='depth', v0=sta.iloc[0]['Station'],\
#                v1=sta.iloc[0]['Latitude'],v2=sta.iloc[0]['Longitude'],v3=sta.iloc[0]['Depth']))


# add rows one at a time, there must be a more efficient way
for k in range(len(sta)):
    addStationRow(k,sta)
    
print(c.execute("SELECT * FROM station limit 50").fetchall())
conn.commit()
conn.close()

# in sqlite, use "PRAGMA table_info('station')" to check column names
# SELECT * FROM station limit 100 to show first 100 entries


In [106]:
#### other version
# following https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_sql.html
# and https://docs.sqlalchemy.org/en/13/core/engines.html

from sqlalchemy import create_engine

dbdir='/Users/ulberg/Documents/GitHub/travelTimeDB/DB' # make sure this directory is created before running
filename=dbdir + '/tt_db2.sqlite'

# # remove database to start (DANGEROUS) - now just use if_exists='replace' for to_sql() call
# if os.path.exists(filename):
#     os.remove(filename)
# else:
#     print("The file does not exist")

# create engine
engine=create_engine('sqlite:///' + filename)

# add station table
sta.to_sql('station', con=engine, if_exists='replace', index=False) # convert dataframe to sqlite db, replace old version (DANGEROUS)

# add event table
keepVar2=['SourceID','Longitude','Latitude','Depth','PNSNid','Longitude-uw','Latitude-uw','Depth-uw','FullTime','EventTime']
df2[keepVar2].to_sql('event', con=engine, if_exists='replace', index=False)

# add observation
keepVar=['SourceID','Station','Phase','Uncertainty','FullArrivalTime','ArrivalTime']
obs_eq[keepVar].to_sql('observation', con=engine, if_exists='replace', index=False)

# look at table
# engine.execute("SELECT * FROM station").fetchall() # return tuples for each row

In [ ]:
# testing
keepVar=['SourceID','Longitude','Latitude','Depth','PNSNid','Longitude-uw','Latitude-uw','Depth-uw','FullTime']
df2[keepVar].head()

In [104]:
obs_eq.head()

,SourceID,Station,Phase,ArrivalTime,Uncertainty,FullArrivalTime
0,4301394p,MO06,P,31.432,0.096,2014-07-01 22:26:31.432
1,4301394p,MM02,P,32.135,0.080,2014-07-01 22:26:32.135
2,4301394p,MO07,P,32.552,0.081,2014-07-01 22:26:32.552
3,4301394p,MN07,P,33.230,0.121,2014-07-01 22:26:33.230
4,4301394p,MJ02,P,33.790,0.092,2014-07-01 22:26:33.790


In [ ]:
engine.execute("SELECT * FROM event").fetchall() # return tuples for each row

Two ways to check table names, while in sqlite command line (open with sqlite3 dbname)
- SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite_%';
or
- .tables

In [ ]:
obs_eq.head()